In [1]:
from fastai.vision.all import *
from utils import *

from fastai.imports import *
from fastai.vision.widgets import *
# to easier work with paths
from pathlib import Path
matplotlib.rc('image', cmap='Greys')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [2]:
import fastbook
fastbook.setup_book()

In [3]:
from fastbook import *
from sklearn.metrics import accuracy_score


In [4]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from IPython.display import Image, display_svg, SVG


from dtreeviz.trees import *
pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [5]:
path = Path()
path.ls()

(#10) [Path('.ipynb_checkpoints'),Path('models'),Path('TytanicClassifier2.ipynb'),Path('TytanicTrainClassifier.html'),Path('TytanicTrainClassifier.ipynb'),Path('submission_nar_titanic.csv'),Path('test.csv'),Path('train.csv'),Path('TytanicClassifier3.ipynb'),Path('submission_nar2_titanic.csv')]

In [6]:
df_test=  pd.read_csv(path/'test.csv')
df_train=  pd.read_csv(path/'train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,...,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",...,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",...,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",...,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",...,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",...,373450,8.0500,NaN,S


In [7]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df_train.isnull().sum().sort_index()/len(df_train)

Age            0.198653
Cabin          0.771044
Embarked       0.002245
Fare           0.000000
Name           0.000000
Parch          0.000000
PassengerId    0.000000
Pclass         0.000000
Sex            0.000000
SibSp          0.000000
Survived       0.000000
Ticket         0.000000
dtype: float64

In [9]:
df_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [10]:
g_train =df_train.columns.to_series().groupby(df_train.dtypes).groups
g_train

{int64: ['PassengerId', 'Survived', 'Pclass', 'SibSp', 'Parch'], float64: ['Age', 'Fare'], object: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']}

In [11]:
dep_var='Survived'

In [12]:
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var)

In [13]:
cat

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [14]:
cont

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [15]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [16]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

to = TabularPandas(df_train, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat,
                   cont_names = cont,
                   y_names=dep_var,
                   splits=splits)

In [17]:
g_train =to.train.xs.columns.to_series().groupby(to.train.xs.dtypes).groups
g_train

{int8: ['Sex', 'Embarked', 'Age_na'], int16: ['Name', 'Ticket', 'Cabin'], float64: ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']}

In [18]:
to.train.xs.Age_na.head()

788    1
525    1
821    1
253    1
374    1
Name: Age_na, dtype: int8

In [19]:
to.train

     PassengerId  Survived    Pclass  Name  ...      Fare  Cabin  Embarked  \
788     1.324792         1  0.824293   209  ... -0.226999      0         3   
525     0.310172         0  0.824293   245  ... -0.477002      0         2   
821     1.452102         1  0.824293   494  ... -0.459214      0         3   
253    -0.739169         0  0.824293   488  ... -0.314232      0         3   
374    -0.272367         0  0.824293   626  ... -0.217253      0         3   
..           ...       ...       ...   ...  ...       ...    ...       ...   
173    -1.047799         0  0.824293   753  ... -0.473591      0         3   
462     0.067126         0 -1.555144   281  ...  0.122419    134         3   
358    -0.334093         1  0.824293   523  ... -0.474483      0         2   
532     0.337177         0  0.824293   237  ... -0.487154      0         1   
344    -0.388103         0 -0.365426   263  ... -0.374662      0         3   

     Age_na  
788       1  
525       1  
821       1  
253    

In [20]:
to.train.xs

,Name,Sex,Ticket,Cabin,...,Age,SibSp,Parch,Fare
788,209,2,549,0,...,-2.182087,0.461591,2.034995,-0.226999
525,245,2,447,0,...,0.861920,-0.462887,-0.464491,-0.477002
821,494,2,270,0,...,-0.178437,-0.462887,-0.464491,-0.459214
253,488,2,537,0,...,0.052753,0.461591,-0.464491,-0.314232
374,626,1,396,0,...,-2.027960,2.310547,0.785252,-0.217253
...,...,...,...,...,...,...,...,...,...
173,753,2,660,0,...,-0.640818,-0.462887,-0.464491,-0.473591
462,281,2,7,134,...,1.362832,-0.462887,-0.464491,0.122419
358,523,1,280,0,...,-0.101374,-0.462887,-0.464491,-0.474483
532,237,2,211,0,...,-0.949072,0.461591,0.785252,-0.487154


In [21]:
to.train.ys.values.ravel()

array([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
     

In [22]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_valid, y_valid = to.valid.xs, to.valid.ys.values.ravel()

In [23]:
def rf(xs, y, n_estimators=40, max_samples=200_000,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True, metrics=accuracy).fit(xs, y)

In [24]:
rnf_classifier= RandomForestRegressor(n_jobs=-1, n_estimators=100)
rnf_classifier.fit(X_train,y_train)

RandomForestRegressor(n_jobs=-1)

In [25]:
y_pred=rnf_classifier.predict(X_valid)


In [26]:
for i in range(len(y_pred)):
    if(y_pred[i] > 0.5):
        y_pred[i] = 1
    else:
        y_pred[i] = 0

In [27]:

accuracy_score(y_pred, y_valid)


0.8370786516853933

In [28]:
test = TabularPandas(df_test, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat,
                   cont_names = cont,
                   )

In [29]:
X_test= test.train.xs

In [30]:
X_test

,Name,Sex,Ticket,Cabin,...,Age,SibSp,Parch,Fare
0,207,2,153,0,...,0.386231,-0.499470,-0.400248,-0.497413
1,404,1,222,0,...,1.371370,0.616992,-0.400248,-0.512278
2,270,2,74,0,...,2.553537,-0.499470,-0.400248,-0.464100
3,409,2,148,0,...,-0.204852,-0.499470,-0.400248,-0.482475
4,179,1,139,0,...,-0.598908,0.616992,0.619896,-0.417491
...,...,...,...,...,...,...,...,...,...
413,354,2,268,0,...,-0.204852,-0.499470,-0.400248,-0.493455
414,284,1,325,23,...,0.740881,-0.499470,-0.400248,1.314435
415,333,2,347,0,...,0.701475,-0.499470,-0.400248,-0.507796
416,385,2,221,0,...,-0.204852,-0.499470,-0.400248,-0.493455


In [31]:
X_train.columns, X_test.columns

(Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age_na', 'PassengerId',
        'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'],
       dtype='object'),
 Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age_na', 'Fare_na',
        'PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'],
       dtype='object'))

In [32]:
X_test= X_test.drop('Fare_na', axis=1)

In [33]:
y_pred=rnf_classifier.predict(X_test)

In [34]:
y_pred= y_pred.astype(int)

In [37]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
     

In [36]:
output= pd.DataFrame({'PassengerId':df_test.PassengerId, 'Survived': y_pred})
# output.to_csv('submission_nar_titanic.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
